In [56]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [110]:
import psycopg2

con = psycopg2.connect(dbname='ADW', user='postgres', password='admin')
cur = con.cursor()

cur.execute('CREATE SCHEMA ADW')
cur.execute("SET search_path TO ADW")


# Create dimension tables
cur.execute('CREATE TABLE dim_date (DATE_ID DATE, annÃ©e VARCHAR(4), mois VARCHAR(2), jour VARCHAR(2))')
cur.execute('ALTER TABLE dim_date ADD CONSTRAINT pk_date PRIMARY KEY (DATE_ID)')

cur.execute('CREATE TABLE dim_temps (TEMPS_ID TIME, heures VARCHAR(2), minutes VARCHAR(2), secondes VARCHAR(2))')
cur.execute('ALTER TABLE dim_temps ADD CONSTRAINT pk_temps PRIMARY KEY (TEMPS_ID)')

cur.execute('CREATE TABLE dim_nature (NATURE_ID VARCHAR(150), description_nature VARCHAR(150))')
cur.execute('ALTER TABLE dim_nature ADD CONSTRAINT pk_nature PRIMARY KEY (NATURE_ID)')

cur.execute('CREATE TABLE dim_equipement (EQUIPEMENT_ID VARCHAR(150), EQU_designation VARCHAR(150), EQU_identifiant VARCHAR(150))')
cur.execute('ALTER TABLE dim_equipement ADD CONSTRAINT pk_equipement PRIMARY KEY (EQUIPEMENT_ID)')

cur.execute('CREATE TABLE dim_operateur (OPERATEUR_ID VARCHAR(150), Username VARCHAR(150))')
cur.execute('ALTER TABLE dim_operateur ADD CONSTRAINT pk_operateur PRIMARY KEY (OPERATEUR_ID)')

cur.execute('CREATE TABLE dim_production (ARTICLE_ID VARCHAR(150), Article_designation VARCHAR(150), Article_libelle VARCHAR(150))')
cur.execute('ALTER TABLE dim_production ADD CONSTRAINT pk_article PRIMARY KEY (ARTICLE_ID)')

cur.execute('CREATE TABLE dim_projet (PROJET_ID VARCHAR(150), PFF VARCHAR(150))')
cur.execute('ALTER TABLE dim_projet ADD CONSTRAINT pk_projet PRIMARY KEY (PROJET_ID)')

cur.execute('CREATE TABLE dim_cle_et_reference (ARTICLEKEY_ID VARCHAR(150), N_Grof VARCHAR(150), N_Of VARCHAR(150))')
cur.execute('ALTER TABLE dim_cle_et_reference ADD CONSTRAINT pk_cle_reference PRIMARY KEY (ARTICLEKEY_ID)')


# Create fact table with foreign key constraints
cur.execute('CREATE TABLE fact_atelier (DATE_ID DATE, TEMPS_ID TIME, NATURE_ID VARCHAR(150), PROJET_ID VARCHAR(150), ARTICLEKEY_ID VARCHAR(150), OPERATEUR_ID VARCHAR(150), EQUIPEMENT_ID VARCHAR(150), WIRE_ID VARCHAR(150), TERMINAL1_ID VARCHAR(150), TERMINAL2_ID VARCHAR(150), SEAL_ID VARCHAR(150), wire_brut integer, wire_good integer, term1_brut integer, term1_good integer, term2_brut integer, term2_good integer, seal_brut integer, seal_good integer, production_requested_pieces integer, produced_pieces integer, scrap_pieces integer, user_requested_pieces integer, total_good_pieces integer, sample_requested_pieces integer)')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_date FOREIGN KEY (DATE_ID) REFERENCES dim_date(DATE_ID) ON DELETE CASCADE')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_temps FOREIGN KEY (TEMPS_ID) REFERENCES dim_temps(TEMPS_ID) ON DELETE CASCADE')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_nature FOREIGN KEY (NATURE_ID) REFERENCES dim_nature(NATURE_ID) ON DELETE CASCADE')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_projet FOREIGN KEY (PROJET_ID) REFERENCES dim_projet(PROJET_ID) ON DELETE CASCADE')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_cle_reference FOREIGN KEY (ARTICLEKEY_ID) REFERENCES dim_cle_et_reference(ARTICLEKEY_ID) ON DELETE CASCADE')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_operateur FOREIGN KEY (OPERATEUR_ID) REFERENCES dim_operateur(OPERATEUR_ID) ON DELETE CASCADE')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_equipement FOREIGN KEY (EQUIPEMENT_ID) REFERENCES dim_equipement(EQUIPEMENT_ID) ON DELETE CASCADE')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_wire FOREIGN KEY (Wire_ID) REFERENCES dim_production(ARTICLE_ID) ON DELETE CASCADE')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_term1 FOREIGN KEY (TERMINAL1_ID) REFERENCES dim_production(ARTICLE_ID) ON DELETE CASCADE')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_term2 FOREIGN KEY (TERMINAL2_ID) REFERENCES dim_production(ARTICLE_ID) ON DELETE CASCADE')
cur.execute('ALTER TABLE fact_atelier ADD CONSTRAINT fk_seal FOREIGN KEY (SEAL_ID) REFERENCES dim_production(ARTICLE_ID) ON DELETE CASCADE')

con.commit()

DuplicateSchema: schema "adw" already exists


In [2]:
import pandas as pd
from datetime import datetime

# Read the CSV file for Producti_530
d_530 = pd.read_csv(r"C:\Users\Pc\Desktop\Work ADW\Producti_530.csv")

# Read the CSV file for Producti_433
d_433 = pd.read_csv(r"C:\Users\Pc\Desktop\Work ADW\Producti_433.csv")


In [4]:
d_530.head()

,Nature,Section,DateTimeStamp,ArticleKey,Job,UserName,Wire,Seal,Terminal,ProductionRequestedPieces,ProductionPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces
0,LearnStarted,Counter,"01/11/2024,09:59:19",57626_-2,"57626_-2, 1",ABDELKARIM,"CU03CR004/F3, 3404150, 3403110","Y2093, 1018, 1010","TMCSFD01, 1019, 1011",NaN,NaN,NaN,NaN,NaN
1,LearnTerminated,Counter,"01/11/2024,09:59:58",57626_-2,"57626_-2, 1",ABDELKARIM,"CU03CR004/F3, 3407665, 3406505","Y2093, 1018, 1010","TMCSFD01, 1019, 1011",NaN,NaN,NaN,NaN,NaN
2,ProductionStarted,Counter,"01/11/2024,10:00:03",57626_-2,"57626_-2, 1",ABDELKARIM,"CU03CR004/F3, 3407665, 3406505","Y2093, 1018, 1010","TMCSFD01, 1019, 1011",3500.0,"1000, 100",NaN,NaN,NaN
3,ProductionInterrupted,Counter,"01/11/2024,10:03:34",57626_-2,"57626_-2, 1",ABDELKARIM,"CU03CR004/F3, 3747290, 3746005","Y2093, 1120, 1111","TMCSFD01, 1121, 1112",3500.0,"1100, 100",0.0,1100.0,NaN
4,ProductionRestarted,Counter,"01/11/2024,10:03:36",57626_-2,"57626_-2, 1",ABDELKARIM,"CU03CR004/F3, 3747290, 3746005","Y2093, 1120, 1111","TMCSFD01, 1121, 1112",3500.0,"1100, 100",0.0,1100.0,NaN


In [6]:
d_433.head()

,Nature,Section,DateTimeStamp,ArticleKey,Job,UserName,Wire,Seal,Terminal,ProductionRequestedPieces,ProductionPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces
0,SampleStarted,Counter,"01/11/2024,06:31:28",57628_-2,"57628_-2, 1",MED AMIN,"CU04CR001, 2904305, 2707141",NaN,"T5556-T, 14124, 13794",NaN,NaN,NaN,NaN,1.0
1,SampleTerminated,Counter,"01/11/2024,06:31:40",57628_-2,"57628_-2, 1",MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,1.0
2,LearnStarted,Counter,"01/11/2024,06:32:19",57628_-2,"57628_-2, 1",MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN
3,LearnAborted,Counter,"01/11/2024,06:32:30",57628_-2,"57628_-2, 1",MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN
4,SampleStarted,Counter,"01/11/2024,06:36:26",57628_-2,"57628_-2, 1",MED AMIN,"CU04CR001, 2904945, 2707141",NaN,"T5556-T, 14126, 13794",NaN,NaN,NaN,NaN,1.0


In [8]:
d_machines = pd.concat([d_433, d_530])

In [10]:
d_machines.head()

,Nature,Section,DateTimeStamp,ArticleKey,Job,UserName,Wire,Seal,Terminal,ProductionRequestedPieces,ProductionPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces
0,SampleStarted,Counter,"01/11/2024,06:31:28",57628_-2,"57628_-2, 1",MED AMIN,"CU04CR001, 2904305, 2707141",NaN,"T5556-T, 14124, 13794",NaN,NaN,NaN,NaN,1.0
1,SampleTerminated,Counter,"01/11/2024,06:31:40",57628_-2,"57628_-2, 1",MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,1.0
2,LearnStarted,Counter,"01/11/2024,06:32:19",57628_-2,"57628_-2, 1",MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN
3,LearnAborted,Counter,"01/11/2024,06:32:30",57628_-2,"57628_-2, 1",MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN
4,SampleStarted,Counter,"01/11/2024,06:36:26",57628_-2,"57628_-2, 1",MED AMIN,"CU04CR001, 2904945, 2707141",NaN,"T5556-T, 14126, 13794",NaN,NaN,NaN,NaN,1.0


In [12]:
d_machines.drop(['Job', 'Section'], axis=1, inplace=True)

In [14]:
d_machines.head()

,Nature,DateTimeStamp,ArticleKey,UserName,Wire,Seal,Terminal,ProductionRequestedPieces,ProductionPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces
0,SampleStarted,"01/11/2024,06:31:28",57628_-2,MED AMIN,"CU04CR001, 2904305, 2707141",NaN,"T5556-T, 14124, 13794",NaN,NaN,NaN,NaN,1.0
1,SampleTerminated,"01/11/2024,06:31:40",57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,1.0
2,LearnStarted,"01/11/2024,06:32:19",57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN
3,LearnAborted,"01/11/2024,06:32:30",57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN
4,SampleStarted,"01/11/2024,06:36:26",57628_-2,MED AMIN,"CU04CR001, 2904945, 2707141",NaN,"T5556-T, 14126, 13794",NaN,NaN,NaN,NaN,1.0


In [16]:
pip install oracledb

Note: you may need to restart the kernel to use updated packages.


In [20]:
import oracledb
username = 'system'
password = 'ADMIN'
dsn = 'localhost:1521/'  # e.g., 'localhost:1521/XEPDB1'

connection = oracledb.connect(user=username, password=password, dsn=dsn)

query = "SELECT * FROM BESOIN_OF_GROF_H"

df = pd.read_sql(query, con=connection)

connection.close()

df

C:\Users\Pc\AppData\Local\Temp\ipykernel_4376\2501662355.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


,BOGH_GROF_NUMERO,BOGH_ORFA_NUMERO,BOGH_NOME_LIGNE,BOGH_EQPT_CODE,BOGH_ORDRE_PRIORITE,BOGH_QTE_BONNE_UE,BOGH_QTE_REBUT_UE,DT_CRE,LOGIN_CRE,DT_MAJ,...,BOGH_SOLDE_FORCE,BOGH_STATUT_AV_SOLDE_FORCE,BOGH_COMMENTAIRE_SOLDE_FORCE,BOGH_ARTKEY,BOGH_COMMENTAIRE_CHANG_STATUT,BOGH_DELETE_APEX,BOGH_DELETE_DT_APEX,BOGH_QUALITY_TOTALBADPIECES,BOGH_TOT_QTE_DECL_MAG_ENC,BOGH_QUALITY_REQPIECES
0,GF2300132,188710,-130,None,NaN,None,None,2024-02-28 18:37:19,GPMEW,2024-07-31 11:42:54,...,N,None,None,45271_-130,None,N,NaT,NaN,NaN,NaN
1,GF2300135,188710,-130,None,NaN,None,None,2024-02-28 18:37:19,GPMEW,2024-07-31 11:42:54,...,N,None,None,45686_-130,None,N,NaT,NaN,NaN,NaN
2,GF2300126,188710,-129,None,NaN,None,None,2024-02-28 18:37:19,GPMEW,2024-07-31 11:42:54,...,N,None,None,44363_-129,None,N,NaT,NaN,NaN,NaN
3,GF2300129,188710,-128,None,NaN,None,None,2024-02-28 18:37:19,GPMEW,2024-07-31 11:42:54,...,N,None,None,44816_-128,None,N,NaT,NaN,NaN,NaN
4,GF2300132,188710,-128,None,NaN,None,None,2024-02-28 18:37:19,GPMEW,2024-07-31 11:42:54,...,N,None,None,45271_-128,None,N,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112291,GF2400094,212300,-495,MACH-CP05,NaN,None,None,2024-08-14 08:03:53,GPMONCEF,2024-09-13 10:11:57,...,N,None,None,55768_-495,None,N,NaT,NaN,NaN,NaN
112292,GF2400094,212300,-500,MACH-CP05,NaN,None,None,2024-08-14 08:03:53,GPMONCEF,2024-09-13 10:11:57,...,N,None,None,55768_-500,None,N,NaT,NaN,NaN,NaN
112293,GF2400094,212300,-507,MACH-CP05,NaN,None,None,2024-08-14 08:03:53,GPMONCEF,2024-09-13 10:11:57,...,N,None,None,55768_-507,None,N,NaT,NaN,NaN,NaN
112294,GF2400094,212300,-512,MACH-CP05,NaN,None,None,2024-08-14 08:03:53,GPMONCEF,2024-09-13 10:11:57,...,N,None,None,55768_-512,None,N,NaT,NaN,NaN,NaN


In [22]:
import oracledb
username = 'system'
password = 'ADMIN'
dsn = 'localhost:1521/'  # e.g., 'localhost:1521/XEPDB1'

connection = oracledb.connect(user=username, password=password, dsn=dsn)

query = "SELECT * FROM BESOIN_OF_H"

hf = pd.read_sql(query, con=connection)

connection.close()

C:\Users\Pc\AppData\Local\Temp\ipykernel_4376\1148561544.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hf = pd.read_sql(query, con=connection)


In [23]:
merged_d_machines = pd.merge(d_machines, df[['BOGH_ARTKEY', 'BOGH_EQPT_CODE']], left_on='ArticleKey', right_on='BOGH_ARTKEY', how='left')
merged_d_machines

,Nature,DateTimeStamp,ArticleKey,UserName,Wire,Seal,Terminal,ProductionRequestedPieces,ProductionPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces,BOGH_ARTKEY,BOGH_EQPT_CODE
0,SampleStarted,"01/11/2024,06:31:28",57628_-2,MED AMIN,"CU04CR001, 2904305, 2707141",NaN,"T5556-T, 14124, 13794",NaN,NaN,NaN,NaN,1.0,57628_-2,MACH-CP03
1,SampleTerminated,"01/11/2024,06:31:40",57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,1.0,57628_-2,MACH-CP03
2,LearnStarted,"01/11/2024,06:32:19",57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN,57628_-2,MACH-CP03
3,LearnAborted,"01/11/2024,06:32:30",57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN,57628_-2,MACH-CP03
4,SampleStarted,"01/11/2024,06:36:26",57628_-2,MED AMIN,"CU04CR001, 2904945, 2707141",NaN,"T5556-T, 14126, 13794",NaN,NaN,NaN,NaN,1.0,57628_-2,MACH-CP03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11846,SampleTerminated,"15/11/2024,22:32:36",57634_-7,HAITHEM,"CU02CR002/F3, 6655, 6105","Y2-GO967067-2, 27, 23","T7-1452665-1, 21, 21",NaN,NaN,NaN,NaN,1.0,57634_-7,MACH-CP07
11847,LearnStarted,"15/11/2024,22:32:44",57634_-7,HAITHEM,"CU02CR002/F3, 6655, 6105","Y2-GO967067-2, 27, 23","T7-1452665-1, 21, 21",NaN,NaN,NaN,NaN,NaN,57634_-7,MACH-CP07
11848,LearnTerminated,"15/11/2024,22:33:28",57634_-7,HAITHEM,"CU02CR002/F3, 6900, 6225","Y2-GO967067-2, 29, 24","T7-1452665-1, 22, 22",NaN,NaN,NaN,NaN,NaN,57634_-7,MACH-CP07
11849,SampleStarted,"15/11/2024,22:33:34",57634_-7,HAITHEM,"CU02CR002/F3, 6900, 6225","Y2-GO967067-2, 29, 24","T7-1452665-1, 22, 22",NaN,NaN,NaN,NaN,NaN,57634_-7,MACH-CP07


In [26]:
merged_d_machines.drop(['BOGH_ARTKEY'], axis=1, inplace=True)

In [28]:
merged_d_machines.head()

,Nature,DateTimeStamp,ArticleKey,UserName,Wire,Seal,Terminal,ProductionRequestedPieces,ProductionPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces,BOGH_EQPT_CODE
0,SampleStarted,"01/11/2024,06:31:28",57628_-2,MED AMIN,"CU04CR001, 2904305, 2707141",NaN,"T5556-T, 14124, 13794",NaN,NaN,NaN,NaN,1.0,MACH-CP03
1,SampleTerminated,"01/11/2024,06:31:40",57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,1.0,MACH-CP03
2,LearnStarted,"01/11/2024,06:32:19",57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN,MACH-CP03
3,LearnAborted,"01/11/2024,06:32:30",57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN,MACH-CP03
4,SampleStarted,"01/11/2024,06:36:26",57628_-2,MED AMIN,"CU04CR001, 2904945, 2707141",NaN,"T5556-T, 14126, 13794",NaN,NaN,NaN,NaN,1.0,MACH-CP03


In [30]:
merged_d_machines[['DATE', 'TIME']] = merged_d_machines['DateTimeStamp'].str.split(',', expand=True)

merged_d_machines['DATE'] = merged_d_machines['DATE'].str.strip()
merged_d_machines['TIME'] = merged_d_machines['TIME'].str.strip()
merged_d_machines.drop(columns=['DateTimeStamp'], inplace=True)

merged_d_machines

,Nature,ArticleKey,UserName,Wire,Seal,Terminal,ProductionRequestedPieces,ProductionPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces,BOGH_EQPT_CODE,DATE,TIME
0,SampleStarted,57628_-2,MED AMIN,"CU04CR001, 2904305, 2707141",NaN,"T5556-T, 14124, 13794",NaN,NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:31:28
1,SampleTerminated,57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:31:40
2,LearnStarted,57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN,MACH-CP03,01/11/2024,06:32:19
3,LearnAborted,57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,NaN,MACH-CP03,01/11/2024,06:32:30
4,SampleStarted,57628_-2,MED AMIN,"CU04CR001, 2904945, 2707141",NaN,"T5556-T, 14126, 13794",NaN,NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:36:26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11846,SampleTerminated,57634_-7,HAITHEM,"CU02CR002/F3, 6655, 6105","Y2-GO967067-2, 27, 23","T7-1452665-1, 21, 21",NaN,NaN,NaN,NaN,1.0,MACH-CP07,15/11/2024,22:32:36
11847,LearnStarted,57634_-7,HAITHEM,"CU02CR002/F3, 6655, 6105","Y2-GO967067-2, 27, 23","T7-1452665-1, 21, 21",NaN,NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:32:44
11848,LearnTerminated,57634_-7,HAITHEM,"CU02CR002/F3, 6900, 6225","Y2-GO967067-2, 29, 24","T7-1452665-1, 22, 22",NaN,NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:33:28
11849,SampleStarted,57634_-7,HAITHEM,"CU02CR002/F3, 6900, 6225","Y2-GO967067-2, 29, 24","T7-1452665-1, 22, 22",NaN,NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:33:34


In [32]:
merged_d_machines[['ProducedPieces', 'ScrapPieces']] = merged_d_machines['ProductionPieces'].str.split(',', expand=True)
merged_d_machines.drop(columns=['ProductionPieces'], inplace=True)

In [34]:
merged_d_machines[['Wire_COD', 'Wire_BRUT', 'Wire_GOOD']] = merged_d_machines['Wire'].str.split(',', expand=True)

merged_d_machines['Wire_COD'] = merged_d_machines['Wire_COD'].str.strip()
merged_d_machines['Wire_BRUT'] = merged_d_machines['Wire_BRUT'].str.strip()
merged_d_machines['Wire_GOOD'] = merged_d_machines['Wire_GOOD'].str.strip()

merged_d_machines

,Nature,ArticleKey,UserName,Wire,Seal,Terminal,ProductionRequestedPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces,BOGH_EQPT_CODE,DATE,TIME,ProducedPieces,ScrapPieces,Wire_COD,Wire_BRUT,Wire_GOOD
0,SampleStarted,57628_-2,MED AMIN,"CU04CR001, 2904305, 2707141",NaN,"T5556-T, 14124, 13794",NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:31:28,NaN,NaN,CU04CR001,2904305,2707141
1,SampleTerminated,57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:31:40,NaN,NaN,CU04CR001,2904625,2707141
2,LearnStarted,57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,MACH-CP03,01/11/2024,06:32:19,NaN,NaN,CU04CR001,2904625,2707141
3,LearnAborted,57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,MACH-CP03,01/11/2024,06:32:30,NaN,NaN,CU04CR001,2904625,2707141
4,SampleStarted,57628_-2,MED AMIN,"CU04CR001, 2904945, 2707141",NaN,"T5556-T, 14126, 13794",NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:36:26,NaN,NaN,CU04CR001,2904945,2707141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11846,SampleTerminated,57634_-7,HAITHEM,"CU02CR002/F3, 6655, 6105","Y2-GO967067-2, 27, 23","T7-1452665-1, 21, 21",NaN,NaN,NaN,1.0,MACH-CP07,15/11/2024,22:32:36,NaN,NaN,CU02CR002/F3,6655,6105
11847,LearnStarted,57634_-7,HAITHEM,"CU02CR002/F3, 6655, 6105","Y2-GO967067-2, 27, 23","T7-1452665-1, 21, 21",NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:32:44,NaN,NaN,CU02CR002/F3,6655,6105
11848,LearnTerminated,57634_-7,HAITHEM,"CU02CR002/F3, 6900, 6225","Y2-GO967067-2, 29, 24","T7-1452665-1, 22, 22",NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:33:28,NaN,NaN,CU02CR002/F3,6900,6225
11849,SampleStarted,57634_-7,HAITHEM,"CU02CR002/F3, 6900, 6225","Y2-GO967067-2, 29, 24","T7-1452665-1, 22, 22",NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:33:34,NaN,NaN,CU02CR002/F3,6900,6225


In [36]:
merged_d_machines[['Seal_COD', 'Seal_BRUT', 'Seal_GOOD']] = merged_d_machines['Seal'].str.split(',', expand=True)

merged_d_machines['Seal_COD'] = merged_d_machines['Seal_COD'].str.strip()
merged_d_machines['Seal_BRUT'] = merged_d_machines['Seal_BRUT'].str.strip()
merged_d_machines['Seal_GOOD'] = merged_d_machines['Seal_GOOD'].str.strip()

merged_d_machines

,Nature,ArticleKey,UserName,Wire,Seal,Terminal,ProductionRequestedPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces,...,DATE,TIME,ProducedPieces,ScrapPieces,Wire_COD,Wire_BRUT,Wire_GOOD,Seal_COD,Seal_BRUT,Seal_GOOD
0,SampleStarted,57628_-2,MED AMIN,"CU04CR001, 2904305, 2707141",NaN,"T5556-T, 14124, 13794",NaN,NaN,NaN,1.0,...,01/11/2024,06:31:28,NaN,NaN,CU04CR001,2904305,2707141,NaN,NaN,NaN
1,SampleTerminated,57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,1.0,...,01/11/2024,06:31:40,NaN,NaN,CU04CR001,2904625,2707141,NaN,NaN,NaN
2,LearnStarted,57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,...,01/11/2024,06:32:19,NaN,NaN,CU04CR001,2904625,2707141,NaN,NaN,NaN
3,LearnAborted,57628_-2,MED AMIN,"CU04CR001, 2904625, 2707141",NaN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,...,01/11/2024,06:32:30,NaN,NaN,CU04CR001,2904625,2707141,NaN,NaN,NaN
4,SampleStarted,57628_-2,MED AMIN,"CU04CR001, 2904945, 2707141",NaN,"T5556-T, 14126, 13794",NaN,NaN,NaN,1.0,...,01/11/2024,06:36:26,NaN,NaN,CU04CR001,2904945,2707141,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11846,SampleTerminated,57634_-7,HAITHEM,"CU02CR002/F3, 6655, 6105","Y2-GO967067-2, 27, 23","T7-1452665-1, 21, 21",NaN,NaN,NaN,1.0,...,15/11/2024,22:32:36,NaN,NaN,CU02CR002/F3,6655,6105,Y2-GO967067-2,27,23
11847,LearnStarted,57634_-7,HAITHEM,"CU02CR002/F3, 6655, 6105","Y2-GO967067-2, 27, 23","T7-1452665-1, 21, 21",NaN,NaN,NaN,NaN,...,15/11/2024,22:32:44,NaN,NaN,CU02CR002/F3,6655,6105,Y2-GO967067-2,27,23
11848,LearnTerminated,57634_-7,HAITHEM,"CU02CR002/F3, 6900, 6225","Y2-GO967067-2, 29, 24","T7-1452665-1, 22, 22",NaN,NaN,NaN,NaN,...,15/11/2024,22:33:28,NaN,NaN,CU02CR002/F3,6900,6225,Y2-GO967067-2,29,24
11849,SampleStarted,57634_-7,HAITHEM,"CU02CR002/F3, 6900, 6225","Y2-GO967067-2, 29, 24","T7-1452665-1, 22, 22",NaN,NaN,NaN,NaN,...,15/11/2024,22:33:34,NaN,NaN,CU02CR002/F3,6900,6225,Y2-GO967067-2,29,24


In [38]:
merged_d_machines.drop(['Wire'], axis=1, inplace=True)

In [40]:
merged_d_machines.drop(['Seal'], axis=1, inplace=True)

In [42]:
import numpy as np
def split_terminal(value):
    # Initialize variables for both terms
    term1_cod = term1_brut = term1_good = np.nan
    term2_cod = term2_brut = term2_good = np.nan

    if pd.notna(value):
        # Check if there is a semicolon separating two terms
        if ';' in value:
            term1, term2 = value.split(';')

            term1_split = term1.split(',')
            if len(term1_split) == 3:
                term1_cod, term1_brut, term1_good = [x.strip() for x in term1_split]

            term2_split = term2.split(',')
            if len(term2_split) == 3:
                term2_cod, term2_brut, term2_good = [x.strip() for x in term2_split]
        else:
            # No semicolon, just one term (Term1) with Term2 as null
            term1_split = value.split(',')
            if len(term1_split) == 3:
                term1_cod, term1_brut, term1_good = [x.strip() for x in term1_split]

    return pd.Series([term1_cod, term1_brut, term1_good, term2_cod, term2_brut, term2_good])

merged_d_machines[['Term1_COD', 'Term1_BRUT', 'Term1_GOOD', 'Term2_COD', 'Term2_BRUT', 'Term2_GOOD']] = merged_d_machines['Terminal'].apply(split_terminal)

merged_d_machines

,Nature,ArticleKey,UserName,Terminal,ProductionRequestedPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces,BOGH_EQPT_CODE,DATE,...,Wire_GOOD,Seal_COD,Seal_BRUT,Seal_GOOD,Term1_COD,Term1_BRUT,Term1_GOOD,Term2_COD,Term2_BRUT,Term2_GOOD
0,SampleStarted,57628_-2,MED AMIN,"T5556-T, 14124, 13794",NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,...,2707141,NaN,NaN,NaN,T5556-T,14124,13794,NaN,NaN,NaN
1,SampleTerminated,57628_-2,MED AMIN,"T5556-T, 14125, 13794",NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,...,2707141,NaN,NaN,NaN,T5556-T,14125,13794,NaN,NaN,NaN
2,LearnStarted,57628_-2,MED AMIN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,MACH-CP03,01/11/2024,...,2707141,NaN,NaN,NaN,T5556-T,14125,13794,NaN,NaN,NaN
3,LearnAborted,57628_-2,MED AMIN,"T5556-T, 14125, 13794",NaN,NaN,NaN,NaN,MACH-CP03,01/11/2024,...,2707141,NaN,NaN,NaN,T5556-T,14125,13794,NaN,NaN,NaN
4,SampleStarted,57628_-2,MED AMIN,"T5556-T, 14126, 13794",NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,...,2707141,NaN,NaN,NaN,T5556-T,14126,13794,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11846,SampleTerminated,57634_-7,HAITHEM,"T7-1452665-1, 21, 21",NaN,NaN,NaN,1.0,MACH-CP07,15/11/2024,...,6105,Y2-GO967067-2,27,23,T7-1452665-1,21,21,NaN,NaN,NaN
11847,LearnStarted,57634_-7,HAITHEM,"T7-1452665-1, 21, 21",NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,...,6105,Y2-GO967067-2,27,23,T7-1452665-1,21,21,NaN,NaN,NaN
11848,LearnTerminated,57634_-7,HAITHEM,"T7-1452665-1, 22, 22",NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,...,6225,Y2-GO967067-2,29,24,T7-1452665-1,22,22,NaN,NaN,NaN
11849,SampleStarted,57634_-7,HAITHEM,"T7-1452665-1, 22, 22",NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,...,6225,Y2-GO967067-2,29,24,T7-1452665-1,22,22,NaN,NaN,NaN


In [44]:
merged_d_machines.drop(['Terminal'], axis=1, inplace=True)

In [46]:
merged_d_machines

,Nature,ArticleKey,UserName,ProductionRequestedPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces,BOGH_EQPT_CODE,DATE,TIME,...,Wire_GOOD,Seal_COD,Seal_BRUT,Seal_GOOD,Term1_COD,Term1_BRUT,Term1_GOOD,Term2_COD,Term2_BRUT,Term2_GOOD
0,SampleStarted,57628_-2,MED AMIN,NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:31:28,...,2707141,NaN,NaN,NaN,T5556-T,14124,13794,NaN,NaN,NaN
1,SampleTerminated,57628_-2,MED AMIN,NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:31:40,...,2707141,NaN,NaN,NaN,T5556-T,14125,13794,NaN,NaN,NaN
2,LearnStarted,57628_-2,MED AMIN,NaN,NaN,NaN,NaN,MACH-CP03,01/11/2024,06:32:19,...,2707141,NaN,NaN,NaN,T5556-T,14125,13794,NaN,NaN,NaN
3,LearnAborted,57628_-2,MED AMIN,NaN,NaN,NaN,NaN,MACH-CP03,01/11/2024,06:32:30,...,2707141,NaN,NaN,NaN,T5556-T,14125,13794,NaN,NaN,NaN
4,SampleStarted,57628_-2,MED AMIN,NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:36:26,...,2707141,NaN,NaN,NaN,T5556-T,14126,13794,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11846,SampleTerminated,57634_-7,HAITHEM,NaN,NaN,NaN,1.0,MACH-CP07,15/11/2024,22:32:36,...,6105,Y2-GO967067-2,27,23,T7-1452665-1,21,21,NaN,NaN,NaN
11847,LearnStarted,57634_-7,HAITHEM,NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:32:44,...,6105,Y2-GO967067-2,27,23,T7-1452665-1,21,21,NaN,NaN,NaN
11848,LearnTerminated,57634_-7,HAITHEM,NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:33:28,...,6225,Y2-GO967067-2,29,24,T7-1452665-1,22,22,NaN,NaN,NaN
11849,SampleStarted,57634_-7,HAITHEM,NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:33:34,...,6225,Y2-GO967067-2,29,24,T7-1452665-1,22,22,NaN,NaN,NaN


In [48]:
d_data = pd.read_excel(r"C:\Users\Pc\Desktop\Projet ADW\Data.xlsx")

In [50]:
df_Equipement = merged_d_machines[["BOGH_EQPT_CODE"]]
merged_df_Equipement = pd.merge(df_Equipement, d_data[['EQPT_CODE', 'EQPT_DESIGNATION', 'EQPT_IDENTIFIANT']], left_on='BOGH_EQPT_CODE', right_on='EQPT_CODE', how='left')
merged_df_Equipement.drop(columns=['EQPT_CODE'], inplace=True)

In [52]:
file_path = r"C:\Users\Pc\Desktop\Projet ADW\Data.xlsx"
d_data2 = pd.read_excel(file_path, sheet_name="Feuil2")

In [58]:
df_wire = merged_d_machines[["Wire_COD"]]
merged_df_wire = pd.merge(df_wire, d_data2[['ARTI_CODE', 'ARTI_DESIGNATION', 'ARTI_LIBELLE_COURT']], left_on='Wire_COD', right_on='ARTI_CODE', how='left')
merged_df_wire.drop(columns=['ARTI_CODE'], inplace=True)

df_term1 = merged_d_machines[["Term1_COD"]]
merged_df_term1 = pd.merge(df_term1, d_data2[['ARTI_CODE', 'ARTI_DESIGNATION', 'ARTI_LIBELLE_COURT']], left_on='Term1_COD', right_on='ARTI_CODE', how='left')
merged_df_term1.drop(columns=['ARTI_CODE'], inplace=True)

df_term2 = merged_d_machines[["Term2_COD"]]
merged_df_term2 = pd.merge(df_term2, d_data2[['ARTI_CODE', 'ARTI_DESIGNATION', 'ARTI_LIBELLE_COURT']], left_on='Term2_COD', right_on='ARTI_CODE', how='left')
merged_df_term2.drop(columns=['ARTI_CODE'], inplace=True)

In [60]:
df_seal = merged_d_machines[["Seal_COD"]]
merged_df_seal = pd.merge(df_seal, d_data2[['ARTI_CODE', 'ARTI_DESIGNATION', 'ARTI_LIBELLE_COURT']], left_on='Seal_COD', right_on='ARTI_CODE', how='left')
merged_df_seal.drop(columns=['ARTI_CODE'], inplace=True)

In [62]:
df_ArticleKey = merged_d_machines[["ArticleKey"]]
merged_df_ArticleKey = pd.merge(df_ArticleKey, df[['BOGH_ARTKEY', 'BOGH_GROF_NUMERO','BOGH_ORFA_NUMERO']], left_on='ArticleKey', right_on='BOGH_ARTKEY', how='left')
merged_df_ArticleKey.drop(columns=['BOGH_ARTKEY'], inplace=True)

In [84]:
merged_d_machines

,Nature,ArticleKey,UserName,ProductionRequestedPieces,UserRequestedPieces,TotalGoodPieces,SampleRequestedPieces,BOGH_EQPT_CODE,DATE,TIME,...,Seal_BRUT,Seal_GOOD,Term1_COD,Term1_BRUT,Term1_GOOD,Term2_COD,Term2_BRUT,Term2_GOOD,Wire_CONS,Wire_CONS_GOOD
0,SampleStarted,57628_-2,MED AMIN,NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:31:28,...,NaN,NaN,T5556-T,14124,13794,NaN,NaN,NaN,0.0,0.0
1,SampleTerminated,57628_-2,MED AMIN,NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:31:40,...,NaN,NaN,T5556-T,14125,13794,NaN,NaN,NaN,320.0,0.0
2,LearnStarted,57628_-2,MED AMIN,NaN,NaN,NaN,NaN,MACH-CP03,01/11/2024,06:32:19,...,NaN,NaN,T5556-T,14125,13794,NaN,NaN,NaN,0.0,0.0
3,LearnAborted,57628_-2,MED AMIN,NaN,NaN,NaN,NaN,MACH-CP03,01/11/2024,06:32:30,...,NaN,NaN,T5556-T,14125,13794,NaN,NaN,NaN,0.0,0.0
4,SampleStarted,57628_-2,MED AMIN,NaN,NaN,NaN,1.0,MACH-CP03,01/11/2024,06:36:26,...,NaN,NaN,T5556-T,14126,13794,NaN,NaN,NaN,320.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11846,SampleTerminated,57634_-7,HAITHEM,NaN,NaN,NaN,1.0,MACH-CP07,15/11/2024,22:32:36,...,27,23,T7-1452665-1,21,21,NaN,NaN,NaN,125.0,120.0
11847,LearnStarted,57634_-7,HAITHEM,NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:32:44,...,27,23,T7-1452665-1,21,21,NaN,NaN,NaN,0.0,0.0
11848,LearnTerminated,57634_-7,HAITHEM,NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:33:28,...,29,24,T7-1452665-1,22,22,NaN,NaN,NaN,245.0,120.0
11849,SampleStarted,57634_-7,HAITHEM,NaN,NaN,NaN,NaN,MACH-CP07,15/11/2024,22:33:34,...,29,24,T7-1452665-1,22,22,NaN,NaN,NaN,0.0,0.0


In [116]:
merged_d_machines['Wire_BRUT'] = pd.to_numeric(merged_d_machines['Wire_BRUT'], errors='coerce').fillna(0)

merged_d_machines['Wire_CONS'] = (
    merged_d_machines.groupby('Wire_COD')['Wire_BRUT']
    .diff()
    .fillna(0)
    .clip(lower=0)  # Remplacer les valeurs négatives par 0
)
merged_d_machines[['Wire_COD', 'Wire_BRUT', 'Wire_CONS']].head(n=21)

,Wire_COD,Wire_BRUT,Wire_CONS
0,CU04CR001,2904305.0,0.0
1,CU04CR001,2904625.0,320.0
2,CU04CR001,2904625.0,0.0
3,CU04CR001,2904625.0,0.0
4,CU04CR001,2904945.0,320.0
5,CU04CR001,2905265.0,320.0
6,CU04CR001,2905265.0,0.0
7,CU04CR001,2905585.0,320.0
8,CU04CR001,2905585.0,0.0
9,CU04CR001,2906205.0,620.0


In [118]:
merged_d_machines['Wire_GOOD'] = pd.to_numeric(merged_d_machines['Wire_GOOD'], errors='coerce').fillna(0)

merged_d_machines['Wire_CONS_GOOD'] = (
    merged_d_machines.groupby('Wire_COD')['Wire_GOOD']
    .diff()
    .fillna(0)
    .clip(lower=0)  # Remplacer les valeurs négatives par 0
)
merged_d_machines[['Wire_COD', 'Wire_GOOD', 'Wire_CONS_GOOD']].head(n=15)

,Wire_COD,Wire_GOOD,Wire_CONS_GOOD
0,CU04CR001,2707141.0,0.0
1,CU04CR001,2707141.0,0.0
2,CU04CR001,2707141.0,0.0
3,CU04CR001,2707141.0,0.0
4,CU04CR001,2707141.0,0.0
5,CU04CR001,2707141.0,0.0
6,CU04CR001,2707141.0,0.0
7,CU04CR001,2707141.0,0.0
8,CU04CR001,2707141.0,0.0
9,CU04CR001,2707141.0,0.0


In [86]:
merged_d_machines['Seal_BRUT'] = pd.to_numeric(merged_d_machines['Seal_BRUT'], errors='coerce').fillna(0)

merged_d_machines['Seal_CONS'] = (
    merged_d_machines.groupby('Seal_COD')['Seal_BRUT']
    .diff()
    .fillna(0)
    .clip(lower=0)  # Remplacer les valeurs négatives par 0
)
merged_d_machines[['Seal_COD', 'Seal_BRUT', 'Seal_CONS']]

,Seal_COD,Seal_BRUT,Seal_CONS
0,NaN,0.0,0.0
1,NaN,0.0,0.0
2,NaN,0.0,0.0
3,NaN,0.0,0.0
4,NaN,0.0,0.0
...,...,...,...
11846,Y2-GO967067-2,27.0,1.0
11847,Y2-GO967067-2,27.0,0.0
11848,Y2-GO967067-2,29.0,2.0
11849,Y2-GO967067-2,29.0,0.0


In [92]:
merged_d_machines['Seal_GOOD'] = pd.to_numeric(merged_d_machines['Seal_GOOD'], errors='coerce').fillna(0)

merged_d_machines['Seal_CONS_GOOD'] = (
    merged_d_machines.groupby('Seal_COD')['Seal_GOOD']
    .diff()
    .fillna(0)
    .clip(lower=0)  # Remplacer les valeurs négatives par 0
)
merged_d_machines[['Seal_COD', 'Seal_GOOD', 'Seal_CONS_GOOD']]

,Seal_COD,Seal_GOOD,Seal_CONS_GOOD
0,NaN,0.0,0.0
1,NaN,0.0,0.0
2,NaN,0.0,0.0
3,NaN,0.0,0.0
4,NaN,0.0,0.0
...,...,...,...
11846,Y2-GO967067-2,23.0,1.0
11847,Y2-GO967067-2,23.0,0.0
11848,Y2-GO967067-2,24.0,1.0
11849,Y2-GO967067-2,24.0,0.0


In [106]:
merged_d_machines['Term1_BRUT'] = pd.to_numeric(merged_d_machines['Term1_BRUT'], errors='coerce').fillna(0)

merged_d_machines['Term1_CONS_BRUT'] = (
    merged_d_machines.groupby('Term1_COD')['Term1_BRUT']
    .diff()
    .fillna(0)
    .clip(lower=0)  # Remplacer les valeurs négatives par 0
)

merged_d_machines['Term1_GOOD'] = pd.to_numeric(merged_d_machines['Term1_GOOD'], errors='coerce').fillna(0)

merged_d_machines['Term1_CONS_GOOD'] = (
    merged_d_machines.groupby('Term1_COD')['Term1_GOOD']
    .diff()
    .fillna(0)
    .clip(lower=0)  # Remplacer les valeurs négatives par 0
)

merged_d_machines['Term2_BRUT'] = pd.to_numeric(merged_d_machines['Term2_BRUT'], errors='coerce').fillna(0)

merged_d_machines['Term2_CONS_BRUT'] = (
    merged_d_machines.groupby('Term2_COD')['Term2_BRUT']
    .diff()
    .fillna(0)
    .clip(lower=0)  # Remplacer les valeurs négatives par 0
)

merged_d_machines['Term2_GOOD'] = pd.to_numeric(merged_d_machines['Term2_GOOD'], errors='coerce').fillna(0)

merged_d_machines['Term2_CONS_GOOD'] = (
    merged_d_machines.groupby('Term2_COD')['Term2_GOOD']
    .diff()
    .fillna(0)
    .clip(lower=0)  # Remplacer les valeurs négatives par 0
)

In [114]:
import pandas as pd
import psycopg2
merged_d_machines["DATE"] = pd.to_datetime(merged_d_machines["DATE"], errors="coerce", format="%d/%m/%Y")
merged_d_machines["TIME"] = pd.to_datetime(merged_d_machines["TIME"], errors="coerce", format="%H:%M:%S").dt.time


con=psycopg2.connect(dbname='ADW',user='postgres',password='admin')
cursor = con.cursor()
cursor.execute("SET search_path TO ADW")

#DIM DATE
for index, row in merged_d_machines.iterrows():
    date = row["DATE"]
    if pd.notna(date):
        jour = date.day
        mois = date.month
        annee = date.year
        date_id = datetime(annee, mois, jour)
        cursor.execute("INSERT INTO dim_date (date_id, annee, mois, jour) VALUES (%s, %s, %s, %s) ON CONFLICT DO NOTHING", (date_id, annee, mois, jour))

#DIM TIME

for index, row in merged_d_machines.iterrows():
    time_value = row["TIME"]
    if pd.notna(time_value):
        # Extract time components
        heure = time_value.hour
        minute = time_value.minute
        seconde = time_value.second

        temps_id = f"{heure:02}:{minute:02}:{seconde:02}"  # Format as HH:MM:SS

        # Insert into the database
        cursor.execute(
            """
            INSERT INTO dim_temps (temps_id, heures, minutes, secondes) 
            VALUES (%s, %s, %s, %s) 
            ON CONFLICT DO NOTHING
            """,
            (temps_id, heure, minute, seconde)
        )
for index, row in merged_d_machines.iterrows():
    nature_id = row["Nature"]  # Get the value of "Nature" from the current row
    description_nature = row["Nature"]  # Assuming the description is the same as the nature ID

    cursor.execute(
        """
        INSERT INTO dim_nature (nature_id, description_nature) 
        VALUES (%s, %s) 
        ON CONFLICT DO NOTHING
        """,
        (nature_id, description_nature)
    )
#DIM EQUIPEMENNT
for index, row in merged_df_Equipement.iterrows():
    Equipement_id = row["BOGH_EQPT_CODE"] 
    Equipement_designation = row["EQPT_DESIGNATION"]
    Equipement_identifiant = row["EQPT_IDENTIFIANT"]

    cursor.execute(
        """
        INSERT INTO dim_equipement (equipement_id, equ_designation, equ_identifiant) 
        VALUES (%s, %s, %s) 
        ON CONFLICT DO NOTHING
        """,
        (Equipement_id, Equipement_designation, Equipement_identifiant)
    )
for index, row in merged_d_machines.iterrows():
    Username = row["UserName"]

    cursor.execute(
        """
        INSERT INTO dim_operateur (operateur_id, username) 
        VALUES (%s, %s) 
        ON CONFLICT DO NOTHING
        """,
        (Username, Username)
    )
for index, row in merged_df_wire.iterrows():
    article_id = row["Wire_COD"]
    Article_designation = row["ARTI_DESIGNATION"]
    Article_libelle = row["ARTI_LIBELLE_COURT"]

    cursor.execute(
        """
        INSERT INTO dim_production (article_id, article_designation, Article_libelle) 
        VALUES (%s, %s, %s) 
        ON CONFLICT DO NOTHING
        """,
        (article_id, Article_designation, Article_libelle)
    )
for index, row in merged_df_term1.iterrows():
    article_id = row["Term1_COD"]
    Article_designation = row["ARTI_DESIGNATION"]
    Article_libelle = row["ARTI_LIBELLE_COURT"]

    cursor.execute(
        """
        INSERT INTO dim_production (article_id, article_designation, Article_libelle) 
        VALUES (%s, %s, %s) 
        ON CONFLICT DO NOTHING
        """,
        (article_id, Article_designation, Article_libelle)
    )
for index, row in merged_df_term2.iterrows():
    article_id = row["Term2_COD"]
    Article_designation = row["ARTI_DESIGNATION"]
    Article_libelle = row["ARTI_LIBELLE_COURT"]

    cursor.execute(
        """
        INSERT INTO dim_production (article_id, article_designation, Article_libelle) 
        VALUES (%s, %s, %s) 
        ON CONFLICT DO NOTHING
        """,
        (article_id, Article_designation, Article_libelle)
    )
for index, row in merged_df_seal.iterrows():
    article_id = row["Seal_COD"]
    Article_designation = row["ARTI_DESIGNATION"]
    Article_libelle = row["ARTI_LIBELLE_COURT"]

    cursor.execute(
        """
        INSERT INTO dim_production (article_id, article_designation, Article_libelle) 
        VALUES (%s, %s, %s) 
        ON CONFLICT DO NOTHING
        """,
        (article_id, Article_designation, Article_libelle)
    )
for index, row in merged_df_ArticleKey.iterrows():
    articlekey_id = row["ArticleKey"]
    n_GROF = row["BOGH_GROF_NUMERO"]
    n_Of = row["BOGH_ORFA_NUMERO"]

    cursor.execute(
        """
        INSERT INTO dim_cle_et_reference (articlekey_id, n_GROF, n_Of) 
        VALUES (%s, %s, %s) 
        ON CONFLICT DO NOTHING
        """,
        (articlekey_id,n_GROF, n_Of)
    )
#Fact Atelier
#DIM Projet
    cursor.execute(
        """
        INSERT INTO dim_projet (projet_id, pff) 
        VALUES (%s, %s) 
        ON CONFLICT DO NOTHING
        """,
        ("", "")
    )


# Fact Atelier
for index, row in merged_d_machines.iterrows():
    date_id = row["DATE"]
    temps_id = row["TIME"]
    nature_id = str(row["Nature"])
    articleKEY_id = str(row["ArticleKey"])
    operateur_id = str(row["UserName"])
    equipement_id = str(row["BOGH_EQPT_CODE"])
    wire_id = row["Wire_COD"]
    term1_id = row["Term1_COD"]
    term2_id = row["Term2_COD"]
    seal_id = row["Seal_COD"]

    # Remplacer les valeurs NaN par None pour laisser vide dans la BD (NULL en PostgreSQL)
    wire_brut = row["Wire_CONS"] if not pd.isna(row["Wire_CONS"]) else None
    wire_good = row["Wire_CONS_GOOD"] if not pd.isna(row["Wire_CONS_GOOD"]) else None
    term1_brut = row["Term1_CONS_BRUT"] if not pd.isna(row["Term1_CONS_BRUT"]) else None
    term1_good = row["Term1_CONS_GOOD"] if not pd.isna(row["Term1_CONS_GOOD"]) else None
    term2_brut = row["Term2_CONS_BRUT"] if not pd.isna(row["Term2_CONS_BRUT"]) else None
    term2_good = row["Term2_CONS_GOOD"] if not pd.isna(row["Term2_CONS_GOOD"]) else None
    seal_brut = row["Seal_CONS"] if not pd.isna(row["Seal_CONS"]) else None
    seal_good = row["Seal_CONS_GOOD"] if not pd.isna(row["Seal_CONS_GOOD"]) else None
    production_requested_pieces = row["ProductionRequestedPieces"] if not pd.isna(row["ProductionRequestedPieces"]) else None
    produced_pieces = row["ProducedPieces"] if not pd.isna(row["ProducedPieces"]) else None
    scrap_pieces = row["ScrapPieces"] if not pd.isna(row["ScrapPieces"]) else None
    user_requested_pieces = row["UserRequestedPieces"] if not pd.isna(row["UserRequestedPieces"]) else None
    total_good_pieces = row["TotalGoodPieces"] if not pd.isna(row["TotalGoodPieces"]) else None
    sample_requested_pieces = row["SampleRequestedPieces"] if not pd.isna(row["SampleRequestedPieces"]) else None

    cursor.execute(
        """
        INSERT INTO fact_atelier (date_id, temps_id, nature_id, projet_id, articlekey_id, operateur_id, equipement_id, wire_id, terminal1_id, terminal2_id, seal_id, wire_brut, wire_good, term1_brut, term1_good, term2_brut, term2_good, seal_brut, seal_good, production_requested_pieces, produced_pieces, scrap_pieces, user_requested_pieces, total_good_pieces, sample_requested_pieces) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
        ON CONFLICT DO NOTHING
        """,
        (date_id, temps_id, nature_id, "", articleKEY_id, operateur_id, equipement_id, wire_id, term1_id, term2_id, seal_id,
         wire_brut, wire_good, term1_brut, term1_good, term2_brut, term2_good, seal_brut, seal_good,
         production_requested_pieces, produced_pieces, scrap_pieces, user_requested_pieces, total_good_pieces, sample_requested_pieces)
    )

con.commit()
cursor.close()
con.close()